In [1]:
import time
import numpy as np
from IPython.core.display import display, HTML

In [2]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def wait(t = 2):
    time.sleep(t)

def pick_a_card(cards, remove = True, manual = False):
    
    if manual:
        return input('name a card to pick:')
    else:
        suit_pick = list(cards.keys())[np.random.randint(0, 100) % len(cards)]
        card_pick = cards[suit_pick][np.random.randint(0, 100) % len(cards[suit_pick])]
        if remove:
            del cards[suit_pick][cards[suit_pick].index(card_pick)]
            if len(cards[suit_pick]) == 0:
                del cards[suit_pick]
        return suit_pick + '_' + card_pick

# blackjack
def calc_points(cards, base_val = 21):
    card_values = {'A' : 11, 'J' : 10, 'Q' : 10, 'K' : 10, '2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6, '7' : 7, '8' : 8, '9' : 9, '10' : 10}
    points = 0
    A_present = False
    for i in range(len(cards)):
        points += card_values[cards[i].split('_')[-1]]
        if cards[i].split('_')[-1] == 'A':
            A_present = True
    if points > base_val and A_present:
        points -= 10
    return points

In [2]:
class Player:
    pl_cnt = 0
    
    def __init__(self, name=''):
        self.money = 500
        Player.pl_cnt += 1
        self.name = name if name.replace(' ', '') != '' else 'Player-%d'%Player.pl_cnt
    
    def new_session(self, card_box):
        self.bj_state = False
        self.cards = [pick_a_card(card_box, True), pick_a_card(card_box, True)]
        self.points = calc_points(self.cards)
        
    def set_bet(self, lowest_bet):
        self.bet = int(input('fund : %d,   enter the initial bet : '%self.money))
        while True:
            if self.bet < lowest_bet:
                self.bet = int(input('invalid !!   lowest required bed is : %d, enter the bet again : '%lowest_bet))
                continue
            if self.bet > self.money:
                self.bet = int(input('invalid !!   fund : %d, enter the bet again : '%self.money))
            else:
                self.money -= self.bet
                break
    
    def add_bet(self):
        bet = int(input('fund : %d,   enter the additional bet : '%self.money))
        while True:
            if bet > self.money:
                bet = int(input('invalid !!   fund : %d, enter the bet again : '%self.money))
            else:
                self.bet += bet
                self.money -= bet
                break
    
    def show_state(self):
        print('your cards : ', self.cards, '\t\t total points : ', self.points)
    
    def check_blackjack(self, base_point):
        if self.points == base_point:
            self.bj_state = True
            print('!!!!!!!! BLACKJACK !!!!!!!!!!')
        return self.bj_state
    
    def hit(self, card_box):
        self.cards += [pick_a_card(card_box, True)]
        self.points = calc_points(self.cards)
        self.show_state()
    
    def update_money(self, win_flag):
        if win_flag == 'w':
            if self.bj_state:
                self.money += (1.5 + 1) * self.bet
            else:
                self.money += self.bet * 2
        elif win_flag == 'd':
            self.money += self.bet
        if self.money <= 0:
            self.money = 0

In [13]:
def card_inc(card_box, n, cnt, shuffle_point, ndeck):
    if cnt + n >= shuffle_point:
        card_box = {
            'heart' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
            'diamond' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
            'club' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
            'spade' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck
        }
        shuffle_point = np.random.randint((ndeck * 52) // 4, (ndeck * 52 * 3) // 4)
        cnt = n
    else:
        cnt += n
    return card_box, cnt, shuffle_point

In [14]:
# initiate
base_point = 21
deal_lbase = 17
lowest_bet = 5
yes = ['y', 'yes']
no = ['n', 'no']
ndeck = 2

init_players = int(input('enter total players : '))
players = np.array([Player(input('enter name - %d : '%(i + 1))) for i in range(init_players)])

# game begins
card_box = {
    'heart' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
    'diamond' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
    'club' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck, 
    'spade' : (['A', 'J', 'Q', 'K'] + ['%d'%i for i in range(2, 11)]) * ndeck
}
card_cnt = 0
shuffle_point = np.random.randint((ndeck * 52) // 2, ndeck * 52)

while True:
    # starting round
    total_players = len(players)
    bjs = [False for i in range(total_players)]
    left_over = [None for i in range(total_players)]
    win_state = ['l' for i in range(total_players)]
    
    # dealer's cards and points
    print(card_box)
    print('dealer is picking two cards...')
    card_box, card_cnt, shuffle_point = card_inc(card_box, 2, card_cnt, shuffle_point, ndeck)
    dealer_cards = [pick_a_card(card_box, True), pick_a_card(card_box, True)]
    dealer_point = calc_points([dealer_cards[0]])
    
    # showing the current points and card of the dealer
    print('-----------------------------------------------------------')
    print('------------dealer 1st card : [', dealer_cards[0], ']   total points : ', dealer_point)
    print('-----------------------------------------------------------\n')
    wait()
    
    for i in range(total_players):
        print(' ****************** PLAYER - %s ****************'%players[i].name)
        # setting initial bet
        players[i].set_bet(lowest_bet)
        wait(.5)
    
    for i in range(total_players):
        print(' ****************** PLAYER - %s ****************'%players[i].name)
        # start session
        print('player is picking two cards...')
        card_box, card_cnt, shuffle_point = card_inc(card_box, 2, card_cnt, shuffle_point, ndeck)
        players[i].new_session(card_box)
        print(card_box)
        players[i].show_state()
        bjs[i] = players[i].check_blackjack(base_point)
        wait()
    
    for i in range(total_players):
        # showing the current points and card of the dealer
        print('-----------------------------------------------------------')
        print('------------dealer 1st card : [', dealer_cards[0], ']   total points : ', dealer_point)
        print('-----------------------------------------------------------\n')
        wait()
        
        print(' ****************** PLAYER - %s ****************'%players[i].name)
        players[i].show_state()
        if bjs[i]:
            left_over[i] = players[i].points
        else:
            while True:
                # stop betting ?
                hit_stand = input('do you want to hit ? (y / n)')
                if hit_stand.lower() in yes:
                    # add bet
                    if players[i].money > 0:
                        dec = input('present bet : %d\t fund : %d\t\t want to add bet ? (y / n)'%(players[i].bet, players[i].money))
                        if dec.lower() in yes:
                            players[i].add_bet()
                            print('bet raised to : %d'%players[i].bet)
                    
                    wait(.5)
                    # new card
                    print('player is picking a new card...')
                    card_box, card_cnt, shuffle_point = card_inc(card_box, 1, card_cnt, shuffle_point, ndeck)
                    players[i].hit(card_box)
                    if players[i].points > base_point:
                        left_over[i] = players[i].points
                        break
                elif hit_stand.lower() in no:
                    left_over[i] = players[i].points
                    break
                else:
                    print('invalid input !!  valid is "y" or "n"')
        print(card_box)
    
    bjs = np.array(bjs)
    win_state = np.array(win_state)
    left_over = np.array(left_over)
#     print(bjs, win_state, left_over)
    
    # win or lose
    # showing the current points and card of the dealer
    dealer_point = calc_points(dealer_cards)
    print('dealer cards : ', dealer_cards, '\t\t total points : ', dealer_point)
    wait()
    
    # dealer has blackjack
    if dealer_point == base_point:
        win_state[bjs] = 'd'
    else:
        # straight win for blackjacks
        win_state[bjs] = 'w'
        
        # dealer will pick
        while True:
            
            if dealer_point >= deal_lbase:
                break
            else:
                print('dealer is picking a new card...')
                dealer_cards += [pick_a_card(card_box, True)]
                dealer_point = calc_points(dealer_cards)
                print('dealer cards : ', dealer_cards, '\t\t total points : ', dealer_point)
                wait()
        
        # checking the situations of winning or losing
        # print(bjs, win_state, left_over)
        # print(((left_over > dealer_point) & (left_over <= base_point)), left_over <= base_point, (left_over == dealer_point))
        if dealer_point > base_point:
            win_state[left_over <= base_point] = 'w'
        else:
            win_state[((left_over > dealer_point) & (left_over <= base_point)) & ~bjs] = 'w'
            win_state[(left_over == dealer_point) & ~bjs] = 'd'
    # print(win_state)
    # settling the money
    for i in range(total_players):
        players[i].update_money(win_state[i])
        if win_state[i] == 'w':
            wstate, gstate = 'wins', 'gains'
        elif win_state[i] == 'd':
            wstate, gstate = 'draws', 'gets back'
        else:
            wstate, gstate = 'loses', 'loses'
        print('"%s" %s, %s bet of %d, current fund: %d'%(players[i].name, wstate, gstate, players[i].bet if not bjs[i] else 1.5 * players[i].bet, players[i].money))
    
    bankrupt = np.array([players[i].money for i in range(total_players)]) < lowest_bet
    wait()
    if bankrupt.sum() > 0:
        print('\\\\\\\\\players ', [players[i].name for i in np.where(bankrupt)[0]], ' eliminated ////////////')
        players = np.delete(players, np.where(bankrupt)[0])
        if len(players) == 0:
            break
        wait()
    
print('###################################################################################')
print('################################## GAME OVER ######################################')

enter total players : 6
enter name - 1 : 
enter name - 2 : 
enter name - 3 : 
enter name - 4 : 
enter name - 5 : 
enter name - 6 : 
{'heart': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'diamond': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'club': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'spade': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10']}
dealer is picking two cards...
-----------------------------------------------------------
------------dealer 1st card : [ club_K ]   total points :  10
-----------------------------------------------------------

 ****************** PLAYER - Player-5 ****************
fund : 500,   enter the i

 ****************** PLAYER - Player-7 ****************
your cards :  ['club_2', 'spade_3'] 		 total points :  5
do you want to hit ? (y / n)y
present bet : 10	 fund : 490		 want to add bet ? (y / n)n
player is picking a new card...
your cards :  ['club_2', 'spade_3', 'club_5'] 		 total points :  10
do you want to hit ? (y / n)y
present bet : 10	 fund : 490		 want to add bet ? (y / n)n
player is picking a new card...
your cards :  ['club_2', 'spade_3', 'club_5', 'spade_Q'] 		 total points :  20
do you want to hit ? (y / n)n
{'heart': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'diamond': ['A', 'J', 'Q', 'K', '2', '5', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'club': ['A', 'Q', '3', '4', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '7', '8', '9', '10'], 'spade': ['A', 'J', '2', '6', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '5', '6', '7', '8',

 ****************** PLAYER - Player-10 ****************
player is picking two cards...
{'heart': ['J', 'K', '2', '3', '4', '5', '6', '8', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'diamond': ['A', 'J', '2', '7', '8', 'A', 'J', 'K', '2', '3', '5', '6', '7', '8', '9', '10'], 'club': ['Q', '3', '4', '9', '10', 'J', 'Q', 'K', '2', '3', '4', '5', '7', '9', '10'], 'spade': ['A', 'J', '2', '6', '7', 'A', 'J', 'Q', 'K', '2', '3', '5', '6', '7', '8', '9', '10']}
your cards :  ['spade_8', 'club_A'] 		 total points :  19
-----------------------------------------------------------
------------dealer 1st card : [ club_A ]   total points :  11
-----------------------------------------------------------

 ****************** PLAYER - Player-5 ****************
your cards :  ['spade_10', 'diamond_9'] 		 total points :  19
do you want to hit ? (y / n)y
player is picking a new card...
your cards :  ['spade_10', 'diamond_9', 'spade_2'] 		 total points :  21
do you want to hit ? (y

 ****************** PLAYER - Player-9 ****************
player is picking two cards...
{'heart': ['J', 'K', '2', '4', '5', '6', '10', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'diamond': ['2', '7', 'A', 'J', 'K', '2', '6', '7', '8', '9', '10'], 'club': ['Q', '3', '4', '9', '10', 'J', 'Q', 'K', '2', '3', '4', '7', '9', '10'], 'spade': ['A', '6', 'A', 'J', 'Q', 'K', '2', '3', '5', '6', '7', '8', '9', '10']}
your cards :  ['diamond_8', 'heart_8'] 		 total points :  16
 ****************** PLAYER - Player-10 ****************
player is picking two cards...
{'heart': ['J', 'K', '2', '4', '5', '6', '10', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'diamond': ['2', '7', 'J', 'K', '2', '6', '7', '8', '9', '10'], 'club': ['Q', '3', '4', '9', '10', 'J', 'Q', 'K', '2', '3', '4', '7', '9', '10'], 'spade': ['A', '6', 'A', 'J', 'Q', 'K', '3', '5', '6', '7', '8', '9', '10']}
your cards :  ['diamond_A', 'spade_2'] 		 total points :  13
-----------------------------------------

-----------------------------------------------------------
------------dealer 1st card : [ spade_9 ]   total points :  9
-----------------------------------------------------------

 ****************** PLAYER - Player-6 ****************
your cards :  ['spade_5', 'club_Q'] 		 total points :  15
do you want to hit ? (y / n)y
present bet : 1200	 fund : 50		 want to add bet ? (y / n)n
player is picking a new card...
your cards :  ['spade_5', 'club_Q', 'club_9'] 		 total points :  24
{'heart': ['J', '2', '4', '5', '10', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'diamond': ['7', '2', '7', '8', '9', '10'], 'club': ['3', '10', 'Q', 'K', '2', '3', '4', '7', '10'], 'spade': ['A', 'Q', 'K', '3', '8', '10']}
-----------------------------------------------------------
------------dealer 1st card : [ spade_9 ]   total points :  9
-----------------------------------------------------------

 ****************** PLAYER - Player-7 ****************
your cards :  ['club_4', 'spade_7'] 		 t

 ****************** PLAYER - Player-9 ****************
your cards :  ['heart_10', 'spade_10'] 		 total points :  20
do you want to hit ? (y / n)n
{'heart': ['4', '5', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'club': ['10', 'Q', 'K', '2', '3', '7', '10'], 'spade': ['A']}
dealer cards :  ['diamond_2', 'diamond_7'] 		 total points :  9
dealer is picking a new card...
dealer cards :  ['diamond_2', 'diamond_7', 'heart_Q'] 		 total points :  19
"Player-6" wins, gains bet of 50, current fund: 100
"Player-7" loses, loses bet of 10, current fund: 1037
"Player-8" draws, gets back bet of 410, current fund: 510
"Player-9" wins, gains bet of 50, current fund: 540
{'heart': ['4', '5', 'J', 'K', '2', '3', '4', '5', '6', '7', '8', '10'], 'club': ['10', 'Q', 'K', '2', '3', '7', '10'], 'spade': ['A']}
dealer is picking two cards...
-----------------------------------------------------------
------------dealer 1st card : [ spade_A ]   total points :  11
-----------------------------------

 ****************** PLAYER - Player-8 ****************
player is picking two cards...
{'heart': ['A', 'J', 'Q', 'K', '3', '7', '8', '9', '10', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'diamond': ['A', 'Q', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'club': ['A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '8', '9', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'spade': ['A', 'J', 'Q', 'K', '2', '3', '4', '7', '8', '9', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10']}
your cards :  ['diamond_K', 'club_10'] 		 total points :  20
-----------------------------------------------------------
------------dealer 1st card : [ spade_5 ]   total points :  5
-----------------------------------------------------------

 ****************** PLAYER - Player-7 ****************
your cards :  ['heart_6', 'heart_5'] 		 total points :  11
do you want to hit ? (y / n)y
player is pick

 ****************** PLAYER - Player-8 ****************
player is picking two cards...
{'heart': ['A', '3', '7', '8', '9', 'A', 'J', 'Q', 'K', '3', '4', '6', '7', '8', '9', '10'], 'diamond': ['A', 'Q', '2', '3', '4', '7', '8', '9', '10', 'A', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10'], 'club': ['A', 'J', 'Q', '3', '5', '6', '9', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '8', '9', '10'], 'spade': ['Q', '3', '4', '7', '9', 'A', 'J', 'Q', 'K', '2', '3', '4', '5', '6', '7', '8', '9', '10']}
your cards :  ['heart_Q', 'spade_K'] 		 total points :  20
-----------------------------------------------------------
------------dealer 1st card : [ diamond_5 ]   total points :  5
-----------------------------------------------------------

 ****************** PLAYER - Player-7 ****************
your cards :  ['club_4', 'heart_K'] 		 total points :  14
do you want to hit ? (y / n)y
player is picking a new card...
your cards :  ['club_4', 'heart_K', 'heart_8'] 		 total points :  22
{'hea